# Batch inference and drift detection

This tutorial leverages a function from the [MLRun Function Hub](https://www.mlrun.org/hub/) to perform [batch inference](https://www.mlrun.org/hub/functions/master/batch_inference_v2/) using a logged model and a new prediction dataset. The function also calculates data drift by comparing the new prediction dataset with the original training set.

Make sure you have reviewed the basics in MLRun [**Quick Start Tutorial**](./01-mlrun-basics.html).

Tutorial steps:
- [**Set up a project**](#setup-project)
- [**View the data**](#view-data)
- [**Log the model with training data**](#log-model)
- [**Import and run the batch inference function**](#run)
- [**Predictions and drift status**](#predictions-and-drift-status)
- [**Examining the drift results in the dashboard**](#examining-the-drift-results-in-the-dashboard)
- [**Next steps**](#next-steps)

## MLRun installation and configuration

Before running this notebook make sure `mlrun` is installed and that you have configured the access to the MLRun service. 

In [1]:
# Install MLRun if not installed, run this only once (restart the notebook after the install !!!)
%pip install mlrun

<a id="setup-project"></a>
## Set up a project

First, import the dependencies and create an [MLRun project](../projects/create-project.html). The project contains all of your models, functions, datasets, etc.:

In [2]:
import mlrun
import pandas as pd

In [3]:
project = mlrun.get_or_create_project("tutorial", context="./", user_project=True)

```{admonition} Note
This tutorial does not focus on training a model. Instead, it starts with a trained model and its corresponding training and prediction dataset.
```

You'll use the following model files and datasets to perform the batch prediction. The model is a [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) from sklearn and the datasets are in `parquet` format.

In [4]:
# We choose the correct model to avoid pickle warnings
import sys

suffix = (
    mlrun.__version__.split("-")[0].replace(".", "_")
    if sys.version_info[1] > 7
    else "3.7"
)

model_path = mlrun.get_sample_path(f"models/batch-predict/model-{suffix}.pkl")
training_set_path = mlrun.get_sample_path("data/batch-predict/training_set.parquet")
prediction_set_path = mlrun.get_sample_path("data/batch-predict/prediction_set.parquet")

<a id="view-data"></a>
## View the data

The training data has 20 numerical features and a binary (0,1) label:

In [5]:
pd.read_parquet(training_set_path).head()

**The prediction data has 20 numerical features, but no label - this is what you will predict:**

In [6]:
pd.read_parquet(prediction_set_path).head()

<a id="log-model"></a>
## Log the model with training data

Next, log the model using MLRun experiment tracking. This is usually done in a training pipeline, but you can also bring in your pre-trained models from other sources. See [Working with data and model artifacts](../training/working-with-data-and-model-artifacts.html) and [Automated experiment tracking](../concepts/auto-logging-mlops.html) for more information.

In this example, you are logging a training set with the model for future comparison, however you can also directly pass in your training set to the batch prediction function.

In [7]:
model_artifact = project.log_model(
    key="model",
    model_file=model_path,
    framework="sklearn",
    training_set=pd.read_parquet(training_set_path),
    label_column="label",
)

In [8]:
# the model artifact unique URI
model_artifact.uri

<a id="run"></a>
## Import and run the batch inference function

Next, import the [batch inference](https://www.mlrun.org/hub/functions/master/batch_inference_v2/) function from the [MLRun Function Hub](https://www.mlrun.org/hub/):

In [9]:
fn = mlrun.import_function("hub://batch_inference_v2")

### Run batch inference

Finally, perform the batch prediction by passing in your model and datasets. In addition, you can trigger the drift analysis batch job on the provided dataset by passing `"trigger_monitoring_job": True`. 

If you do perform drift analysis, a new model endpoint record is generated. Model endpoint is a unique MLRun entity that includes statistics and important details about your model and function. You can perform the drift analysis on an existing model endpoint, but you need to make sure that you don't mix unrelated datasets that could affect the final drift analysis process. In general, it's recommended to perform the drift analysis on a new model endpoint to avoid possible analysis conflicts.

See the corresponding [batch inference example notebook](https://github.com/mlrun/functions/blob/master/batch_inference_v2/batch_inference_v2.ipynb) for an exhaustive list of other parameters that are supported:

In [10]:
run = project.run_function(
    fn,
    inputs={
        "dataset": prediction_set_path,
    },
    params={
        "model_path": model_artifact.uri,
        "perform_drift_analysis": True,
        "trigger_monitoring_job": True,
    },
)

<a id="predictions-and-drift-status"></a>
## Predictions and drift status

These are the batch predictions on the prediction set from the model:

In [11]:
run.artifact("prediction").as_df().head()

There is also a drift table plot that compares the drift between the training data and prediction data per feature:

In [ ]:
run.artifact("drift_table_plot").show()

![Drift Table Plot](./_static/images/drift_table_plot.png)

Finally, you also get a numerical drift metric and boolean flag denoting whether or not data drift is detected:

In [13]:
run.status.results

In [14]:
# Data/concept drift per feature
import json

json.loads(run.artifact("features_drift_results").get())

## Examining the drift results in the dashboard
<a id="iguazio-platform"></a>

This section reviews the main charts and statistics that can be found on the platform dashboard. See [Model monitoring overview](../monitoring/model-monitoring-deployment.html) to learn more about the available model monitoring features and how to use them.

Before analyzing the results in the visual dashboards, run another batch infer job, but this time with a lower drift threshold, to get a drifted result. The drift decision rule is the value per-feature mean of the [Total Variance Distance (TVD) and Hellinger distance](../monitoring/model-monitoring-deployment.html#common-terminology) scores. By default, the threshold is 0.7 but you can modify it through the batch infer process. As seen above, the drift result in this case was ~0.3. Reduce the threshold value to 0.2 in the following run to generate the drifted result:

In [16]:
run = project.run_function(
    fn,
    inputs={
        "dataset": prediction_set_path,
    },
    params={
        "model_path": model_artifact.uri,
        "perform_drift_analysis": True,
        "trigger_monitoring_job": True,
        "model_endpoint_name": "drifted-model-endpoint",
        "model_endpoint_drift_threshold": 0.2,
        "model_endpoint_possible_drift_threshold": 0.1,
    },
)

Now you can observe the drift result:

In [17]:
run.status.results

### Model Endpoints

In the Projects page > [Model endpoint summary list](../monitoring/model-monitoring-deployment.html#model-endpoint-summary-list), you can see the new two model endpoints, including their drift status:

![Model Endpoints Summary List](./_static/images/batch_infer_list_model_endpoints.png)

You can zoom into one of the model endpoints to get an [overview](../monitoring/model-monitoring-deployment.html#model-endpoint-overview) about the selected endpoint, including the calculated statistical drift metrics:

![Model Endpoint Overview](./_static/images/batch_infer_overview_model_endpoint.png)

Press [Features Analysis](../monitoring/model-monitoring-deployment.html#model-features-analysis) to see details of the drift analysis in a table format with each feature in the selected model on its own line, including the predicted label:

![Model Endpoint Feature Analysis](./_static/images/batch_infer_feature_analysis.png)

## Next steps

In a production setting, you probably want to incorporate this as part of a larger pipeline or application.

For example, if you use this function for the prediction capabilities, you can pass the `prediction` output as the input to another pipeline step, store it in an external location like S3, or send to an application or user.

If you use this function for the drift detection capabilities, you can use the `drift_status` and `drift_metrics` outputs to automate further pipeline steps, send a notification, or kick off a re-training pipeline.